In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA


In [6]:
import os
OMP_NUM_THREADS=1

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:


# Set random seed for reproducibility
np.random.seed(30)

# Load the data
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns")
    return df

# Data preprocessing function
def preprocess_data(df):
    # Make a copy to avoid modifying the original dataframe
    data = df.copy()
    
    # Check for missing values
    print("\nMissing values per column:")
    print(data.isnull().sum())
    
    # Separate numerical and categorical columns
    numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = ['key', 'mode']
    
    # Remove 'popularity' if it's highly imbalanced (based on our analysis)
    if 'popularity' in numeric_features:
        numeric_features.remove('popularity')
    
    # Create preprocessing pipelines
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    # Apply preprocessing
    X = preprocessor.fit_transform(data)
    
    print(f"\nPreprocessed data shape: {X.shape}")
    
    # Return both the preprocessed data and the feature names
    return X, numeric_features, categorical_features, preprocessor

# Function to determine optimal number of clusters using silhouette score
def find_optimal_clusters(X, max_clusters=10, method='kmeans'):
    silhouette_scores = []
    
    for n_clusters in range(2, max_clusters + 1):
        if method == 'kmeans':
            model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        else:  # Gaussian Mixture Model (EM)
            model = GaussianMixture(n_components=n_clusters, random_state=42, n_init=10)
            
        cluster_labels = model.fit_predict(X)
        
        # Calculate silhouette score
        silhouette_avg = silhouette_score(X, cluster_labels)
        silhouette_scores.append(silhouette_avg)
        
        print(f"For n_clusters = {n_clusters}, the silhouette score is {silhouette_avg:.3f}")
    
    # Find the optimal number of clusters
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    
    # Plot silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(range(2, max_clusters + 1), silhouette_scores, 'o-')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Score for {method.upper()} clustering')
    plt.grid(True)
    plt.savefig(f'optimal_clusters_{method}.png')
    plt.close()
    
    return optimal_clusters

# K-means clustering
def kmeans_clustering(X, n_clusters):
    print(f"\nPerforming K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X)
    
    return kmeans, clusters

# Expectation Maximization (Gaussian Mixture Model)
def em_clustering(X, n_components):
    print(f"\nPerforming EM clustering with {n_components} components...")
    gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=10)
    clusters = gmm.fit_predict(X)
    
    return gmm, clusters

# Function to visualize and store clusters without dimension reduction
def visualize_clusters(X, clusters, method_name, original_data, preprocessor):
    # Add cluster assignments to original data
    original_data_with_clusters = original_data.copy()
    original_data_with_clusters[f'{method_name}_cluster'] = clusters
    
    # Since we're not using PCA, we'll plot a different visualization
    # We'll use a parallel coordinates plot for the most important features
    
    # Select important features for display
    important_features = [
        'danceability_%', 'energy_%', 'acousticness_%', 
        'instrumentalness_%', 'valence_%', 'bpm'
    ]
    important_features = [f for f in important_features if f in original_data.columns]
    
    # Create a copy of the data with the cluster column
    plot_data = original_data.copy()
    plot_data[f'{method_name}_cluster'] = clusters
    
    # Create a parallel coordinates plot
    plt.figure(figsize=(15, 8))
    pd.plotting.parallel_coordinates(
        plot_data.sample(min(500, len(plot_data))),  # Sample to avoid overcrowding
        f'{method_name}_cluster',
        cols=important_features,
        colormap='viridis'
    )
    plt.title(f'Parallel Coordinates Plot - {method_name} Clustering')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'cluster_visualization_{method_name}.png')
    plt.close()
    
    return original_data_with_clusters, None, None


Loaded dataset with 952 rows and 23 columns

Missing values per column:
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
popularity               0
dtype: int64

Preprocessed data shape: (952, 33)


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


For n_clusters = 2, the silhouette score is 0.250
For n_clusters = 3, the silhouette score is 0.208
For n_clusters = 4, the silhouette score is 0.120


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


For n_clusters = 5, the silhouette score is 0.104
For n_clusters = 6, the silhouette score is 0.098
For n_clusters = 7, the silhouette score is 0.095
For n_clusters = 8, the silhouette score is 0.079


...

For n_clusters = 9, the silhouette score is 0.013


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_

For n_clusters = 10, the silhouette score is -0.004

Optimal number of components for EM: 2

Performing EM clustering with 2 components...


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_


Analyzing kmeans clusters...

Cluster sizes:
kmeans_cluster
0    770
1    182
Name: count, dtype: int64

Cluster centroids for key features:
                danceability_%   energy_%  acousticness_%  instrumentalness_%  \
kmeans_cluster                                                                  
0                    67.593506  64.133766       27.133766            1.580519   
1                    64.406593  64.868132       26.846154            1.593407   

                valence_%         bpm  
kmeans_cluster                         
0               51.451948  122.450649  
1               51.214286  122.989011  

Analyzing em clusters...

Cluster sizes:
em_cluster
0    675
1    277
Name: count, dtype: int64

Cluster centroids for key features:
            danceability_%   energy_%  acousticness_%  instrumentalness_%  \
em_cluster                                                                  
0                68.325926  64.871111       25.943704            0.000000   
1       

In [7]:

# Analyze clusters
def analyze_clusters(df_with_clusters, numeric_features, categorical_features, method_name):
    print(f"\nAnalyzing {method_name} clusters...")
    cluster_column = f'{method_name}_cluster'
    
    # Get unique clusters
    clusters = df_with_clusters[cluster_column].unique()
    
    # Basic cluster statistics
    cluster_sizes = df_with_clusters[cluster_column].value_counts().sort_index()
    print("\nCluster sizes:")
    print(cluster_sizes)
    
    # Calculate cluster centroids for numeric features
    print("\nCluster centroids for key features:")
    cluster_means = df_with_clusters.groupby(cluster_column)[numeric_features].mean()
    
    # Select important features for display
    important_features = [
        'danceability_%', 'energy_%', 'acousticness_%', 
        'instrumentalness_%', 'valence_%', 'bpm'
    ]
    important_features = [f for f in important_features if f in numeric_features]
    
    print(cluster_means[important_features])
    
    # Create a radar chart for each cluster
    categories = important_features
    N = len(categories)
    
    # Create a figure for the radar chart
    fig = plt.figure(figsize=(15, 12))
    
    # Compute angle for each category
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]
    
    # Normalize cluster means for radar chart
    scaler = StandardScaler()
    normalized_means = pd.DataFrame(
        scaler.fit_transform(cluster_means[important_features]),
        index=cluster_means.index,
        columns=important_features
    )
    
    # Create radar charts
    for i, cluster_id in enumerate(normalized_means.index):
        ax = fig.add_subplot(2, (len(clusters) + 1) // 2, i + 1, polar=True)
        
        # Get values for this cluster
        values = normalized_means.loc[cluster_id].values.tolist()
        values += values[:1]
        
        # Plot values
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {cluster_id}')
        ax.fill(angles, values, alpha=0.25)
        
        # Fix axis to go in the right order and start at the top
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        
        # Draw axis lines for each angle and label
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories)
        
        # Set title
        ax.set_title(f'Cluster {cluster_id} (n={cluster_sizes[cluster_id]})', size=11, y=1.1)
    
    plt.tight_layout()
    plt.savefig(f'cluster_profiles_{method_name}.png')
    plt.close()
    
    return cluster_means


In [8]:

# Main function
def main(file_path, max_clusters=10):
    # Load data
    df = load_data(file_path)
    
    # Preprocess data
    X, numeric_features, categorical_features, preprocessor = preprocess_data(df)
    
    # Find optimal number of clusters for KMeans
    k_optimal = find_optimal_clusters(X, max_clusters=max_clusters, method='kmeans')
    print(f"\nOptimal number of clusters for KMeans: {k_optimal}")
    
    # Perform KMeans clustering
    kmeans_model, kmeans_clusters = kmeans_clustering(X, k_optimal)
    
    # Find optimal number of components for EM
    em_optimal = find_optimal_clusters(X, max_clusters=max_clusters, method='em')
    print(f"\nOptimal number of components for EM: {em_optimal}")
    
    # Perform EM clustering
    gmm_model, gmm_clusters = em_clustering(X, em_optimal)
    
    # Visualize clusters (without dimension reduction)
    df_with_kmeans, _, _ = visualize_clusters(X, kmeans_clusters, 'kmeans', df, preprocessor)
    df_with_em, _, _ = visualize_clusters(X, gmm_clusters, 'em', df, preprocessor)
    
    # Combine cluster assignments
    df_with_clusters = df.copy()
    df_with_clusters['kmeans_cluster'] = kmeans_clusters
    df_with_clusters['em_cluster'] = gmm_clusters
    
    # Analyze clusters
    kmeans_profiles = analyze_clusters(df_with_clusters, numeric_features, categorical_features, 'kmeans')
    em_profiles = analyze_clusters(df_with_clusters, numeric_features, categorical_features, 'em')
    
    # Save results
    df_with_clusters.to_csv('spotify_with_clusters.csv', index=False)
    print("\nResults saved to 'spotify_with_clusters.csv'")
    
    # Compare KMeans and EM clustering
    print("\nComparing KMeans and EM clustering:")
    contingency_table = pd.crosstab(df_with_clusters['kmeans_cluster'], df_with_clusters['em_cluster'])
    print(contingency_table)
    
    # Plot comparison
    plt.figure(figsize=(10, 6))
    sns.heatmap(contingency_table, annot=True, cmap='viridis', fmt='d')
    plt.title('Comparison of KMeans and EM Clustering')
    plt.xlabel('EM Cluster')
    plt.ylabel('KMeans Cluster')
    plt.tight_layout()
    plt.savefig('cluster_comparison.png')
    plt.close()
    
    return df_with_clusters, kmeans_model, gmm_model

# Run the main function if executed directly
if __name__ == "__main__":
    file_path = 'Input_data/spotify_processed.csv'
    df_with_clusters, kmeans_model, gmm_model = main(file_path)

Loaded dataset with 952 rows and 23 columns

Missing values per column:
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
popularity               0
dtype: int64

Preprocessed data shape: (952, 33)
For n_clusters = 2, the silhouette score is 0.250
For n_clusters = 3, the silhouette score is 0.208


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


For n_clusters = 4, the silhouette score is 0.120
For n_clusters = 5, the silhouette score is 0.104


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


For n_clusters = 6, the silhouette score is 0.098
For n_clusters = 7, the silhouette score is 0.095


...

For n_clusters = 9, the silhouette score is 0.013


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_

For n_clusters = 10, the silhouette score is -0.004

Optimal number of components for EM: 2

Performing EM clustering with 2 components...


C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python312\site-packages\sklearn\cluster\_


Analyzing kmeans clusters...

Cluster sizes:
kmeans_cluster
0    770
1    182
Name: count, dtype: int64

Cluster centroids for key features:
                danceability_%   energy_%  acousticness_%  instrumentalness_%  \
kmeans_cluster                                                                  
0                    67.593506  64.133766       27.133766            1.580519   
1                    64.406593  64.868132       26.846154            1.593407   

                valence_%         bpm  
kmeans_cluster                         
0               51.451948  122.450649  
1               51.214286  122.989011  

Analyzing em clusters...

Cluster sizes:
em_cluster
0    675
1    277
Name: count, dtype: int64

Cluster centroids for key features:
            danceability_%   energy_%  acousticness_%  instrumentalness_%  \
em_cluster                                                                  
0                68.325926  64.871111       25.943704            0.000000   
1       